## Configure PUDL
The `.pudl.yml` configuration file tells PUDL where to look for data. Uncomment the next cell and run it if you're on our 2i2c JupyterHub.

In [1]:
#!cp ~/shared/shared-pudl.yml ~/.pudl.yml

In [2]:
# import the necessary packages
%load_ext autoreload
%autoreload 2

import pandas as pd
import sqlalchemy as sa
import pudl

# Connecting to the PUDL Databases

This notebook will walk you through several ways of pulling data out of the Public Utility Data Liberation (PUDL)
project databases and into [Pandas](https://pandas.pydata.org/) Dataframes for analysis and visualization.

This notebook assumes you have a development version of the [PUDL Python package](https://github.com/catalyst-cooperative/pudl) installed, and a complete PUDL database available locally, in the location expected by the Python package.

If you have any questions or feedback you can:
* [Create an issue](https://github.com/catalyst-cooperative/pudl-tutorials/issues) in the GitHub repo for our tutorials, or
* Contact the team at: pudl@catalyst.coop

## Direct SQLite Access
Much of the PUDL data is published as [SQLite database files](https://www.sqlite.org/index.html). These are relational databases generally intended for use by a single user at a time. If you're already familiar with databases and SQL in Python, you can access them just like you would any other. [Support for SQLite](https://docs.python.org/3/library/sqlite3.html) is built into the Python standard libraries, and the popular [SQLAlchemy](https://www.sqlalchemy.org) Python package also has extensive support for SQLite.  Here's one in-depth resource on using Python, SQLite and SQLAlchemy together: [Data Management with Python, SQLite, and SQLAlchemy](https://realpython.com/python-sqlite-sqlalchemy/)

For the rest of these tutorials, we're going to assume you want to get the data into Pandas as quickly as possible for interactive work.


## Database Normalization
The data in the PUDL database has been extensively deduplicated, [normalized](https://en.wikipedia.org/wiki/Database_normalization) and generally organized according to best practices of [tidy data](https://tidyr.tidyverse.org/articles/tidy-data.html) in order to ensure that it is internally self-consistent and free of errors. As a result, you'll often need to combine information from more than one table to make it readable or to get all the information you need for your analysis in one place. We've built some tools to do this automatically, which we'll get to below.

## Locate the PUDL DB file
Each SQLite database is stored within a single file. To access the data, you need to know where that file is. With the location of the file, you can create an [SQLAlchemy connection engine](https://docs.sqlalchemy.org/en/13/core/engines.html), which Pandas will use to read data out of the database. PUDL stores its data in a directory structure generally organized by file format. We store the paths to those directories and the SQLAlchemy database URLs in a Python dictionary that's usually called `pudl_settings`. Note that  a URL is just a path to a file that could be either local (on your computer) or remote (on someone else's computer). The following command will construct that `pudl_settings` dictionary based on some directory paths stored in the `.pudl.yml` file in your home directory. Printing out the dictionary contents you can see where PUDL will look for various resources.

In [3]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_settings

{'pudl_in': '/home/zane/code/catalyst/pudl-work',
 'data_dir': '/home/zane/code/catalyst/pudl-work/data',
 'settings_dir': '/home/zane/code/catalyst/pudl-work/settings',
 'pudl_out': '/home/zane/code/catalyst/pudl-work',
 'sqlite_dir': '/home/zane/code/catalyst/pudl-work/sqlite',
 'parquet_dir': '/home/zane/code/catalyst/pudl-work/parquet',
 'datapkg_dir': '/home/zane/code/catalyst/pudl-work/datapkg',
 'notebook_dir': '/home/zane/code/catalyst/pudl-work/notebook',
 'ferc1_db': 'sqlite:////home/zane/code/catalyst/pudl-work/sqlite/ferc1.sqlite',
 'pudl_db': 'sqlite:////home/zane/code/catalyst/pudl-work/sqlite/pudl.sqlite'}

## The SQLAlchemy Connection Engine
The `sqlalchemy.create_engine()` function takes a database URL and creates an Engine that knows how to interact with the database. It can do things like list out the names of all the tables in the database.

In [4]:
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
# see all the tables inside of the database
pudl_engine.table_names()

['assn_gen_eia_unit_epa',
 'assn_plant_id_eia_epa',
 'boiler_fuel_eia923',
 'boiler_generator_assn_eia860',
 'boilers_entity_eia',
 'coalmine_eia923',
 'energy_source_eia923',
 'ferc_accounts',
 'ferc_depreciation_lines',
 'fuel_ferc1',
 'fuel_receipts_costs_eia923',
 'fuel_type_aer_eia923',
 'fuel_type_eia923',
 'generation_eia923',
 'generation_fuel_eia923',
 'generators_eia860',
 'generators_entity_eia',
 'ownership_eia860',
 'plant_in_service_ferc1',
 'plant_unit_epa',
 'plants_eia',
 'plants_eia860',
 'plants_entity_eia',
 'plants_ferc1',
 'plants_hydro_ferc1',
 'plants_pudl',
 'plants_pumped_storage_ferc1',
 'plants_small_ferc1',
 'plants_steam_ferc1',
 'prime_movers_eia923',
 'purchased_power_ferc1',
 'transport_modes_eia923',
 'utilities_eia',
 'utilities_eia860',
 'utilities_entity_eia',
 'utilities_ferc1',
 'utilities_pudl',
 'utility_plant_assn']

# Reading data with `pandas.read_sql()`
The [pandas.read_sql()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html) method is the simplest way to pull data from an SQL database into a dataframe. You can give it an SQL statement to execute, or just the name of a table to read in its entirety.

## Read a whole table
Reading an entire table all at once is easy. It isn't very memory efficient but there's less than 1 GB of data in the PUDL database, so in most cases this is a fine option. Once you've had a chance to poke around at the whole table a bit, you can select the data that's actually of interest out of it for your analysis or visualization.

You can also explore the contents of the database interactively online at https://data.catalyst.coop if you want to familiarize yourself with its contents in a more graphical way first.

In [5]:
generation_df = pd.read_sql("generation_eia923", pudl_engine)
generation_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479352 entries, 0 to 479351
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  479352 non-null  int64         
 1   plant_id_eia        479352 non-null  int64         
 2   generator_id        479352 non-null  object        
 3   report_date         479352 non-null  datetime64[ns]
 4   net_generation_mwh  454504 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 41.8 MB


In [6]:
generation_df.sample(10)

,id,plant_id_eia,generator_id,report_date,net_generation_mwh
156509,156510,3130,FB1,2013-06-01,184609.00
367,368,50,5,2009-08-01,5883.00
255786,255787,6177,CO2,2015-07-01,264659.00
384437,384438,3,A1ST,2018-06-01,122069.00
205198,205199,6195,ST2,2014-11-01,109528.00
112803,112804,1252,7,2012-04-01,-716.00
382653,382654,58007,G2,2017-10-01,3939.17
338932,338933,609,3A,2017-05-01,82343.00
287729,287730,7,1,2016-06-01,7823.00
238254,238255,126,ST3,2015-07-01,22437.00


## Select specific data using SQL
If you're familiar with SQL, and you already know what subset of the data you want to pull out of the database, you can give Pandas an SQL statement directly, along with the `pudl_engine`, and it will put the results of the SQL statement into a dataframe for you.

For example, the following statement sums the nameplate capacities of generators by power plant, for every generator that reported a capacity in the EIA 860 in 2019, excluding those in Alaska and Hawaii. It sorts the results by capacity with the biggest plants first, and only returns the biggest 1000 plants.

[Compare with the results from our online database](https://data.catalyst.coop/pudl?sql=select%0D%0A++plants.plant_id_eia%2C%0D%0A++plants.plant_name_eia%2C%0D%0A++SUM%28gens.capacity_mw%29+as+plant_capacity_mw%2C%0D%0A++latitude%2C%0D%0A++longitude%0D%0Afrom%0D%0A++generators_eia860+as+gens%0D%0Ajoin%0D%0A++plants_entity_eia+as+plants%0D%0Awhere%0D%0A++plants.plant_id_eia+%3D+gens.plant_id_eia%0D%0A++and+gens.report_date+%3D+%222019-01-01%22%0D%0A++and+plants.state+not+in+%28%22HI%22%2C+%22AK%22%29%0D%0Agroup+by%0D%0A++plants.plant_id_eia%0D%0Aorder+by%0D%0A++plant_capacity_mw+desc).

This method is much faster and less memory intensive than reading whole tables, but it requires familiarity with SQL and the structure of the database. If you have a solid state disk and plenty of RAM, reading whole tables into memory is generally plenty fast, and shouldn't run into memory constraints.

In [7]:
example_sql = """
SELECT
  plants.plant_id_eia,
  plants.plant_name_eia,
  SUM(gens.capacity_mw) AS plant_capacity_mw,
  latitude,
  longitude
FROM
  generators_eia860 AS gens
JOIN
  plants_entity_eia AS plants
WHERE
  plants.plant_id_eia = gens.plant_id_eia
  AND gens.report_date = "2019-01-01"
  AND plants.state not in ("HI", "AK")
GROUP BY
  plants.plant_id_eia
ORDER BY
  plant_capacity_mw DESC
LIMIT 1000;
"""
big_plants_df = pd.read_sql(example_sql, pudl_engine)
big_plants_df

,plant_id_eia,plant_name_eia,plant_capacity_mw,latitude,longitude
0,6163,Grand Coulee,6809.0,47.957511,-118.977323
1,6043,Martin,6071.5,27.053600,-80.562800
2,628,Crystal River,5303.7,28.965600,-82.697700
3,649,Vogtle,4630.0,33.142700,-81.762500
4,56407,West County Energy Center,4263.0,26.698600,-80.374700
...,...,...,...,...,...
995,389,El Centro Hybrid,438.3,32.802222,-115.540000
996,118,Saguaro,435.5,32.551700,-111.300000
997,63113,Southern Bighorn Solar Hybrid,435.0,36.304793,-114.472803
998,56163,KUCC,434.5,40.711900,-112.122500


## The SQLAlchemy expression language
SQLAlchemy provides a Python API for building complex SQL queries, and `pandas.read_sql()` can accept these query objects in place of the SQL statement written out by hand as above. [See the SQLAlchemy documentation for more details](https://docs.sqlalchemy.org/en/13/core/tutorial.html).

# Read tables using the PUDL output layer
Early on in the development of the PUDL database, we found that we were frequently joining the same tables together, and calculating the same derived values in Pandas during our interactive analyses. So we wrote some code to do that work automatically and uniformly. We call this the PUDL Output Layer. It brings in fields like plant and utility names from their home tables, so you have more than just the numeric ID to go by, caches dataframes internally for re-use, and can do some time series aggregation.

These outputs are "denormalized" -- meaning that data will be duplicated in different output tables, and they will contain derived values that don't represent unique information. This structure isn't good inside a database, but it's great for interactive use.

The 2nd notebook in this tutorial is all about the `PudlTabl` objects, which we usually name `pudl_out`, but here is a quick preview.

If you want to access de-normalized tables, we've built an access methodology that saves access methods for most denormalized tables in PUDL and analysis build ontop of PUDL tables. There is a whole other notebook that covers the output tables so if you want more info on that.

## Create a PudlTabl output object
The tabular output object needs to know what PUDL database it's connecting to (via the `pudl_engine` argument), and optionally, what time frequency it should aggregate tables on.

In [8]:
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine)

## Construct denormalized dataframes
The `PudlTabl` object, called `pudl_out` here, has a bunch of methods corresponding to individual tables within the database. They typically use abbreviated names. Hitting `Tab` will show you a preview the available methods.

The `gen_eia923()` method corresponds to the `generation_eia923` table in the database, which details the monthly net generation from each generator reporting on the EIA Form 923.

Note: if you re-run the cell, it will complete almost instantly, because the dataframe has been cached inside the `pudl_out` object for later use.

In [9]:
%%time
gen_eia923 = pudl_out.gen_eia923()
gen_eia923.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 476052 entries, 0 to 476051
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   report_date         476052 non-null  object 
 1   plant_id_eia        476052 non-null  Int64  
 2   plant_id_pudl       476052 non-null  Int64  
 3   plant_name_eia      476052 non-null  object 
 4   utility_id_eia      476052 non-null  Int64  
 5   utility_id_pudl     476052 non-null  Int64  
 6   utility_name_eia    476052 non-null  object 
 7   generator_id        476052 non-null  object 
 8   net_generation_mwh  451368 non-null  float64
dtypes: Int64(4), float64(1), object(4)
memory usage: 38.1+ MB
CPU times: user 14.3 s, sys: 755 ms, total: 15 s
Wall time: 18.2 s


In [10]:
gen_eia923.sample(10)

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,net_generation_mwh
22747,2009-08-01,10398,3229,ISG Cleveland Works,9454,518,ISG Cleveland Inc,GENB,1152.0
192993,2014-10-01,2322,116,Clark (NVE),13407,204,Nevada Power Co,9,-195.0
234451,2015-08-01,7,206,Gadsden,195,18,Alabama Power Co,2,-367.0
418218,2018-07-01,55221,4455,Wrightsville Power Facility,807,529,Arkansas Electric Coop Corp,G9,14858.0
298941,2016-10-01,3804,493,Possum Point,19876,349,Dominion Virginia Power,6A,306520.0
340703,2017-12-01,1843,1892,Shiras,11701,1092,Marquette City of,3,25921.0
64125,2010-10-01,54338,4063,Rio Grande Valley Sugar Growers,16098,3006,"Rio Grande Valley Sugar Growers, Inc.",GENA,5.0
360840,2017-01-01,50498,3687,Capitol District Energy Center,2956,5579,Capital District Energy Center,GTG,182.0
401214,2018-07-01,10091,3104,Total Energy Facilities,11217,2301,Los Angeles County Sanitation,GEN4,2540.0
212737,2014-02-01,50359,3612,Sloss Industries Corp,17330,3694,"ERP Compliant Coke, LLC",9,0.0


## Compare with the normalized DB table
The denormalized version of the table above includes fields like `utility_name_eia923` and `plant_name_eia923` and `plant_id_pudl` which are all useful, but aren't fundamentally part of this table -- they can all be looked up in other tables based on the value of `plant_id_eia` found in the original `generation_eia923` table, so storing them in this table would mean duplicating data.  You can see what the original table looks like below.

Note also that since we're going back to the database directly rather than accessing the cached dataframe within the `pudl_out` object, this query will take a few seconds to run, just like the first time we read the table using `pudl_out` above.

In [11]:
%%time
gen_eia923_normalized = pd.read_sql("generation_eia923", pudl_engine)
gen_eia923_normalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479352 entries, 0 to 479351
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  479352 non-null  int64         
 1   plant_id_eia        479352 non-null  int64         
 2   generator_id        479352 non-null  object        
 3   report_date         479352 non-null  datetime64[ns]
 4   net_generation_mwh  454504 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 18.3+ MB
CPU times: user 2.43 s, sys: 51.3 ms, total: 2.48 s
Wall time: 3.56 s


In [12]:
gen_eia923_normalized.sample(10)

,id,plant_id_eia,generator_id,report_date,net_generation_mwh
119358,119359,2876,2,2012-07-01,95685.0
383026,383027,58159,CGT3,2017-11-01,3099.0
52282,52283,4048,3,2010-11-01,0.0
291771,291772,874,6,2016-04-01,0.0
419922,419923,55132,GTG1,2018-07-01,91226.0
81025,81026,2324,4,2011-02-01,77498.0
91005,91006,7710,ST3,2011-10-01,117028.0
314881,314882,50305,GEN9,2016-02-01,3666.0
9736,9737,2067,2,2009-05-01,NaN
477943,477944,58557,GT2,2019-08-01,90501.0


# FERC Form 1: Here Be Dragons
You might have noticed up above that there were actually two SQLite database URLs in the `pudl_settings` object... One for PUDL, and another for FERC Form 1.

In [13]:
pudl_settings

{'pudl_in': '/home/zane/code/catalyst/pudl-work',
 'data_dir': '/home/zane/code/catalyst/pudl-work/data',
 'settings_dir': '/home/zane/code/catalyst/pudl-work/settings',
 'pudl_out': '/home/zane/code/catalyst/pudl-work',
 'sqlite_dir': '/home/zane/code/catalyst/pudl-work/sqlite',
 'parquet_dir': '/home/zane/code/catalyst/pudl-work/parquet',
 'datapkg_dir': '/home/zane/code/catalyst/pudl-work/datapkg',
 'notebook_dir': '/home/zane/code/catalyst/pudl-work/notebook',
 'ferc1_db': 'sqlite:////home/zane/code/catalyst/pudl-work/sqlite/ferc1.sqlite',
 'pudl_db': 'sqlite:////home/zane/code/catalyst/pudl-work/sqlite/pudl.sqlite'}

## FERC Form 1: Direct vs. PUDL
The PUDL database contains a tiny fraction of the data available in the original FERC Form 1 -- we have only taken the time to clean a handful of the FERC tables. The original FERC Form 1 data is often very messy and poorly organized. However, if you need to access one of the original 113 tables that we haven't integrated yet, they're all available, going back to 1994. The original tables are only accessible via direct queries (either using SQL or pulling whole tables) from the original FERC Form 1 database, so you'll have to use the `pandas.read_sql()` methods outlined above.

If there are particular tables within the FERC Form 1 that you think are important to get cleaned up, let us know so we can prioritize them going forward!

In [14]:
ferc1_engine = sa.create_engine(pudl_settings["ferc1_db"])
# see all the tables inside of the database
ferc1_engine.table_names()

['f1_106_2009',
 'f1_106a_2009',
 'f1_106b_2009',
 'f1_208_elc_dep',
 'f1_231_trn_stdycst',
 'f1_324_elc_expns',
 'f1_325_elc_cust',
 'f1_331_transiso',
 'f1_338_dep_depl',
 'f1_397_isorto_stl',
 'f1_398_ancl_ps',
 'f1_399_mth_peak',
 'f1_400_sys_peak',
 'f1_400a_iso_peak',
 'f1_429_trans_aff',
 'f1_acb_epda',
 'f1_accumdepr_prvsn',
 'f1_accumdfrrdtaxcr',
 'f1_adit_190_detail',
 'f1_adit_190_notes',
 'f1_adit_amrt_prop',
 'f1_adit_other',
 'f1_adit_other_prop',
 'f1_allowances',
 'f1_allowances_nox',
 'f1_audit_log',
 'f1_bal_sheet_cr',
 'f1_capital_stock',
 'f1_cash_flow',
 'f1_cmmn_utlty_p_e',
 'f1_cmpinc_hedge',
 'f1_cmpinc_hedge_a',
 'f1_co_directors',
 'f1_codes_val',
 'f1_col_lit_tbl',
 'f1_comp_balance_db',
 'f1_construction',
 'f1_control_respdnt',
 'f1_cptl_stk_expns',
 'f1_csscslc_pcsircs',
 'f1_dacs_epda',
 'f1_dscnt_cptl_stk',
 'f1_edcfu_epda',
 'f1_elc_op_mnt_expn',
 'f1_elc_oper_rev_nb',
 'f1_elctrc_erg_acct',
 'f1_elctrc_oper_rev',
 'f1_electric',
 'f1_email',
 'f1_envrn